In [68]:
MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
#prompt = [{"role": "user", "content": ""}]
loops = 1
#temperature = 1.5
#file = f"{MODEL}/single message/2" #alphabet/5"
file = "best chat"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
import copy

client = OpenAI()
df_path = os.getcwd()
chat = False


def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages
def load (path):
    with open(path, "r") as f:
        completions = pd.read_json(f, lines=True)
    return completions

In [12]:
#completions = load("completions/gpt-3.5-turbo/alphabet/5")
#completions

In [36]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(os.path.join(df_path,file), "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])
    print(completions)

Empty DataFrame
Columns: [Response, Eval, Data, Instruction]
Index: []


In [37]:
prompt

[{'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Hello! How can I assist you today?'}]

In [39]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        #temperature = temperature,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [66]:
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [44]:
completions = load("completions/gpt-3.5-turbo/1")

In [57]:
completions_old = load("completions/gpt-3.5-turbo/alphabet/6")

In [58]:
completions = pd.concat([completions, completions_old],ignore_index=True)

In [80]:
level = completions.iloc[400:500]

In [73]:
level

,Response,Eval,Data,Instruction
100,Hello! I'm here to help with any questions or ...,,{'Completion': {'id': 'chatcmpl-9wVm1lJJz83ono...,"[{'role': 'user', 'content': 'Hello! How can I..."
101,Hello! I'm here to answer any questions you ma...,,{'Completion': {'id': 'chatcmpl-9wVm2zeUTXULrQ...,"[{'role': 'user', 'content': 'Hello! How can I..."
102,Hello! I'm just here to help with any question...,,{'Completion': {'id': 'chatcmpl-9wVm371Ft0IgMj...,"[{'role': 'user', 'content': 'Hello! How can I..."
103,I'm an AI digital assistant here to help you w...,,{'Completion': {'id': 'chatcmpl-9wVm37GeVe3Pt3...,"[{'role': 'user', 'content': 'Hello! How can I..."
104,Hello! I am an AI assistant here to help answe...,,{'Completion': {'id': 'chatcmpl-9wVm4yX65qO8TO...,"[{'role': 'user', 'content': 'Hello! How can I..."
...,...,...,...,...
195,Hello! I am here to help answer any questions ...,,{'Completion': {'id': 'chatcmpl-9wVnXdh9q3hKls...,"[{'role': 'user', 'content': 'Hello! How can I..."
196,Hello! I am an AI digital assistant here to he...,,{'Completion': {'id': 'chatcmpl-9wVnYokWwtpB5Q...,"[{'role': 'user', 'content': 'Hello! How can I..."
197,Hello! I'm here to assist you with any questio...,,{'Completion': {'id': 'chatcmpl-9wVnZZxn3qLKLj...,"[{'role': 'user', 'content': 'Hello! How can I..."
198,Hi there! I was just wondering if you could pr...,,{'Completion': {'id': 'chatcmpl-9wVna05St8DYKj...,"[{'role': 'user', 'content': 'Hello! How can I..."


In [24]:
prompt = completions.iloc[n]["Instruction"]

In [25]:
prompt

[{'role': 'assistant', 'content': ''},
 {'role': 'user', 'content': 'Hello! How can I assist you today?'}]

In [76]:
n = 160
completions.iloc[n]

Response       Hello! Can you provide me with information on ...
Eval                                        First alphabetically
Data           {'Completion': {'id': 'chatcmpl-9wVn0rcvWu1BR6...
Instruction    [{'role': 'user', 'content': 'Hello! How can I...
Name: 160, dtype: object

In [75]:
tag = "First alphabetically"
completions.loc[n,"Eval"] = tag
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")
selected_response = completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Response"]
if chat:
    messages = copy.deepcopy(completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Instruction"])
    messages.append({"role": "assistant", "content": selected_response_response})
    prompt = transform_roles(messages)
else:
    prompt = [{"role": "user", "content": selected_response}]
print(prompt)

[{'role': 'user', 'content': 'Hello! Can you provide me with information on the latest technology trends?'}]
